In [ ]:
!pip install --upgrade datasets fsspec transformers

In [ ]:
# import datasets, tokenizer, model, trainer, training arguments

from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments



hi


In [ ]:
# -- 1) load dataset
dataset = load_dataset("imdb")

In [ ]:
dataset

In [ ]:
train_dataset = dataset["train"].select(range(1000)) # get 1000 for train
test_dataset = dataset["test"].select(range(200)) # get 200 for test

In [ ]:
# -- 2) tokanize

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# tokenize function - tokenization, padding, truncation adn max length

def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)


In [ ]:
# preprocess function - apply tokenization, remove column text(save memory), format to torch 

def preprocess(ds):
    ds = ds.map(tokenize_fn, barched=True, remove_columns=["text"])
    ds = ds.rename_column("label", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds


In [ ]:
train_dataset = preprocess(train_dataset) # preprocess the train dataset

In [ ]:
test_dataset = preprocess(test_dataset) # preprocess the test dataset

In [ ]:
# -- 3) model

# initialize model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


In [ ]:
# #Classifier head trainable rahega by default

# for param in model.bert.parameters():
#     param.requires_grad = False  # Freeze BERT encoder

# #Unfreeze last 2 encoder layers

# for layer in model.bert.encoder.layer[-2:]:
#     for param in layer.parameters():
#         param.requires_grad = True

In [ ]:
# -- 4) training arguments

In [ ]:
# training arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-finetuned-imdb",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)

In [ ]:
# -- 5) trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
# -- 6) train
trainer.train()

In [ ]:
# !tensorboard --logdir=./logs

In [ ]:
trainer.save_model("./bert-finetuned-imdb")

In [ ]:
tokenizer.save_pretrained("./bert-finetuned-imdb")

In [ ]:
# -- 7) evaluate
metrics = trainer.evaluate()

In [ ]:
print(metrics)

In [ ]:
# -- 8) prediction